In [2]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, power_transform, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)

In [4]:
X_train = pd.read_csv('0_X_train.csv', index_col='Id')
X_valid = pd.read_csv('1_X_valid.csv', index_col='Id')
X_test  = pd.read_csv('2_X_test.csv', index_col='Id')

y_train = pd.read_csv('0_y_train.csv', index_col='Id')
y_valid = pd.read_csv('1_y_valid.csv', index_col='Id')
y_test  = pd.read_csv('2_y_test.csv', index_col='Id')

num_vars = ['age', 'time_spent', 'banner_views', 'banner_views_old', 'days_elapsed_old', 'X4']

In [27]:
transformers = [
    ('power_transform', PowerTransformer(standardize = True), num_vars)  # Power transformation and Standardization
]

std_num = ColumnTransformer(transformers=transformers, remainder='passthrough')

In [28]:
pipe_LDA = Pipeline([
    ('std_num', std_num),
    ("lda", LinearDiscriminantAnalysis())])

In [29]:
# Fit pipeline
pipe_LDA.fit(X_train, y_train) # Fit it to the training data 

# Predict on training data
y_train_pred = pipe_LDA.predict(X_train) # Predict the class labels for the training data
acc = accuracy_score(y_train, y_train_pred) # Calculate the accuracy of the predictions 
print("LDA --- Accuracy on training data:", np.round(acc, 3))

# Predict on valid data
y_valid_pred = pipe_LDA.predict(X_valid) # Predict the class labels for the valid data
acc = accuracy_score(y_valid, y_valid_pred) # Calculate the accuracy of the predictions 
print("LDA --- Accuracy on valid data:", np.round(acc, 3))

# Predict on test data
y_test_pred = pipe_LDA.predict(X_test) # Predict the class labels for the test data
acc = accuracy_score(y_test, y_test_pred) # Calculate the accuracy of the predictions 
print("LDA --- Accuracy on test data:", np.round(acc, 3))

LDA --- Accuracy on training data: 0.819
LDA --- Accuracy on valid data: 0.798
LDA --- Accuracy on test data: 0.821


c:\Users\emann\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
# Define model evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Evaluate model
scores = cross_val_score(pipe_LDA, X_train , y_train, scoring='accuracy', cv=cv, n_jobs=-1)

# Summarize result
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# https://machinelearningmastery.com/linear-discriminant-analysis-with-python/
# Running the example evaluates the Linear Discriminant Analysis algorithm on the synthetic dataset and reports the average accuracy across the three repeats of 10-fold cross-validation.

Mean Accuracy: 0.816 (0.012)


In [85]:
# Tune LDA Hyperparameters
LDA = LinearDiscriminantAnalysis()

lda_param_grid = {"solver" : ['svd', 'lsqr', 'eigen'],
              "tol" : [0.0001,0.0002,0.0003]}

gsLDA = GridSearchCV(LDA, param_grid = lda_param_grid, cv=cv,
                     scoring="accuracy", n_jobs= -1, verbose = 1)

gsLDA.fit(X_train,y_train)
LDA_best = gsLDA.best_estimator_

# Best score
gsLDA.best_score_


Fitting 30 folds for each of 9 candidates, totalling 270 fits


c:\Users\emann\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
84 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\emann\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\emann\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py", line 599, in fit
    self._solve_eigen(
  File "c:\Users\emann\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py", line 440, in _solve_eigen
    evals, evecs = linalg.eigh(Sb, Sw)
  File "c:\Users\emann\anaconda3

0.7989698566291807

In [16]:
# Define pipeline
pipe_QDA = Pipeline(
    [("power", PowerTransformer()), 
     ("qda", QuadraticDiscriminantAnalysis())])

In [17]:
# Fit pipeline
pipe_QDA.fit(X_train, y_train) # Fit it to the training data 

c:\Users\emann\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\emann\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Pipeline(steps=[('power', PowerTransformer()),
                ('qda', QuadraticDiscriminantAnalysis())])

In [18]:
# Add noise before fitting again
def add_noise(X, scale):
    """
    DataFrame double -> ndarray
    produce DataFrame adding Gaussian with standard deviation = `scale` to each column of `X`.
    """
    # Transform to ndarray
    X_arr = X.to_numpy()

    # Get shape
    n, d = X_arr.shape

    # Add Gaussian noise
    X_arr = X_arr + np.random.normal(scale = scale, size = (n, d))

    # Back to DataFrame
    df = pd.DataFrame(X_arr, columns = ["X" + str(i) for i in range(1, d + 1)])

    # Return df
    return df

X_train_noise = add_noise(X_train, scale = 0.1) # Add random noise to the input and controls the amount of the noise added
pipe_QDA.fit(X_train_noise, y_train) # Fit it to the training data 

c:\Users\emann\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('power', PowerTransformer()),
                ('qda', QuadraticDiscriminantAnalysis())])

_____________________________________________________________________________________________________________________________________________________________________

https://www.datasklr.com/select-classification-methods/linear-and-quadratic-discriminant-analysis

In [5]:
import sklearn
from sklearn.preprocessing import PolynomialFeatures

# Create interaction terms (interaction of each regressor pair + polynomial)
#Interaction terms need to be created in both the test and train datasets
interaction2 = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False, order='C') #second degree
interaction3 = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False, order='C') #third degree

# Training
X_train_2 = pd.DataFrame(interaction2.fit_transform(X_train), columns=interaction2.get_feature_names(input_features=X_train.columns))
X_train_3 = pd.DataFrame(interaction3.fit_transform(X_train), columns=interaction3.get_feature_names(input_features=X_train.columns))
X_train_2.head()

# Validation
X_valid_2 = pd.DataFrame(interaction2.fit_transform(X_valid), columns=interaction2.get_feature_names(input_features=X_valid.columns))
X_valid_3 = pd.DataFrame(interaction3.fit_transform(X_valid), columns=interaction3.get_feature_names(input_features=X_valid.columns))
X_valid_2.head()

# Test
X_test_2 = pd.DataFrame(interaction2.fit_transform(X_test), columns=interaction2.get_feature_names(input_features=X_test.columns))
X_test_3 = pd.DataFrame(interaction3.fit_transform(X_test), columns=interaction3.get_feature_names(input_features=X_test.columns))
X_test_2.head()


c:\Users\emann\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\emann\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\emann\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\emann\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_fe

,age,education,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,...,job_student^2,job_student job_teacher,job_student job_technology,job_student job_unemployed,job_teacher^2,job_teacher job_technology,job_teacher job_unemployed,job_technology^2,job_technology job_unemployed,job_unemployed^2
0,39.0,2.0,0.0,9.0,2.0,5.75,2.0,2.0,183.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,38.0,2.0,0.0,14.0,5.0,14.20,2.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,36.0,2.0,0.0,12.0,2.0,20.45,2.0,1.0,183.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,25.0,0.0,0.0,7.0,7.0,5.30,1.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30.0,2.0,0.0,4.0,6.0,9.20,1.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#############################
## Normalize all datasets 
#############################

power = PowerTransformer()


# Standardize the training sets: 1st, 2nd and 3rd order polynomials
X_train=pd.DataFrame(power.fit_transform(X_train), columns=X_train.columns)
X_train_2=pd.DataFrame(power.fit_transform(X_train_2), columns=X_train_2.columns)
X_train_3=pd.DataFrame(power.fit_transform(X_train_3), columns=X_train_3.columns)

# Standardize the validation sets: 1st, 2nd and 3rd order polynomials
X_valid=pd.DataFrame(power.fit_transform(X_valid), columns=X_valid.columns)
X_valid_2=pd.DataFrame(power.fit_transform(X_valid_2), columns=X_valid_2.columns)
X_valid_3=pd.DataFrame(power.fit_transform(X_valid_3), columns=X_valid_3.columns)

# Standardize the test sets: 1st, 2nd and 3rd order polynomials
X_test=pd.DataFrame(power.fit_transform(X_test), columns=X_test.columns)
X_test_2=pd.DataFrame(power.fit_transform(X_test_2), columns=X_test_2.columns)
X_test_3=pd.DataFrame(power.fit_transform(X_test_3), columns=X_test_3.columns)


c:\Users\emann\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
c:\Users\emann\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
c:\Users\emann\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
c:\Users\emann\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
c:\Users\emann\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
c:\Users\emann\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning:

In [7]:
################################
## Deal with multicollinearity
################################

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

#1st order polynomial ######################
x_temp_train1 = sm.add_constant(X_train)
vif_train1 = pd.DataFrame()
vif_train1["VIF Factor"] = [variance_inflation_factor(x_temp_train1.values, i) for i in range(x_temp_train1.values.shape[1])]
vif_train1["features"] = x_temp_train1.columns
pd.set_option('display.max_rows', 300)
print(vif_train1.round(1))


# Identify all variables wit VIF less then 5 and keep
vif_train1_a=vif_train1[vif_train1["VIF Factor"]<5.0]  # print(vif2.round(1))

feat_list=vif_train1_a["features"].tolist()  #save desired features to list
feat_list.remove(feat_list[0])
print(feat_list)

X_train=X_train[feat_list] #keep features on feature list only, drop all other features for train
X_valid=X_valid[feat_list] #keep features on feature list only, drop all other features for valid



    VIF Factor               features
0          1.0                  const
1          1.9                    age
2          1.8              education
3          1.1                 device
4          1.0                    day
5          1.1                  month
6          1.0             time_spent
7          1.1           banner_views
8        205.4       banner_views_old
9        561.8       days_elapsed_old
10         1.0                     X1
11         1.0                     X2
12         1.2                     X3
13         1.1                     X4
14         inf       marital_divorced
15         inf        marital_married
16         inf         marital_single
17         inf    outcome_old_failure
18         inf         outcome_old_na
19         inf      outcome_old_other
20         inf    outcome_old_success
21         inf       job_entrepreneur
22         inf          job_freelance
23         inf        job_housekeeper
24         inf  job_industrial_worker
25         i

c:\Users\emann\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [8]:
#2nd order polynomial ####################
x_temp_train2 = sm.add_constant(X_train_2)
vif_train2 = pd.DataFrame()
vif_train2["VIF Factor"] = [variance_inflation_factor(x_temp_train2.values, i) for i in range(x_temp_train2.values.shape[1])]
vif_train2["features"] = x_temp_train2.columns
pd.set_option('display.max_rows', 300)
#print(vif_train1.round(1))

vif_train2_a=vif_train2[vif_train2["VIF Factor"]<5.0]
#print(vif2.round(1))

feat_list2=vif_train2_a["features"].tolist()
feat_list2.remove(feat_list2[0])
print(feat_list2)

X_train_2=X_train_2[feat_list2] #keep features on feature list only, drop all other features for train
X_valid_2=X_valid_2[feat_list2] #keep features on feature list only, drop all other features for valid
X_valid_2

c:\Users\emann\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


['device X2', 'device X3', 'X1 X2', 'X1 X3', 'X2 X3']


,device X2,device X3,X1 X2,X1 X3,X2 X3
0,0.0,-0.126036,-0.054656,-0.266407,-0.082138
1,0.0,-0.126036,-0.054656,-0.266407,-0.082138
2,0.0,-0.126036,-0.054656,-0.266407,-0.082138
3,0.0,-0.126036,-0.054656,-0.266407,-0.082138
4,0.0,-0.126036,-0.054656,-0.266407,-0.082138
...,...,...,...,...,...
1338,0.0,-0.126036,-0.054656,3.753650,-0.082138
1339,0.0,-0.126036,-0.054656,3.753650,-0.082138
1340,0.0,-0.126036,-0.054656,-0.266407,-0.082138
1341,0.0,-0.126036,-0.054656,-0.266407,-0.082138


In [9]:
#3rd order polynomial ####################
x_temp_train3 = sm.add_constant(X_train_3)
vif_train3 = pd.DataFrame()
vif_train3["VIF Factor"] = [variance_inflation_factor(x_temp_train3.values, i) for i in range(x_temp_train3.values.shape[1])]
vif_train3["features"] = x_temp_train3.columns
pd.set_option('display.max_rows', 300)
#print(vif_train3.round(1))

vif_train3_a=vif_train3[vif_train3["VIF Factor"]<5.0]
#print(vif3.round(1))

feat_list3=vif_train3_a["features"].tolist()
feat_list3.remove(feat_list3[0])
print(feat_list3)

X_train_3=X_train_3[feat_list3] #keep features on feature list only, drop all other features for train
X_valid_3=X_valid_3[feat_list3]   #keep features on feature list only, drop all other features for valid

KeyboardInterrupt: 

In [ ]:
# Default LDA model without any tuning - base metric
LDA_model_default = LinearDiscriminantAnalysis()
LDA_model_default.fit(X_train, y_train)
y_pred_LDA_default =LDA_model_default.predict(X_test)